In [3]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Copy our H2 function from the previous notebook
def create_simple_h2_hamiltonian(distance=0.74):
    """Create H2 Hamiltonian using known coefficients"""
    distances = np.array([0.5, 0.6, 0.7, 0.74, 0.8, 0.9, 1.0, 1.2, 1.5, 2.0])
    energies = np.array([-1.0557, -1.1135, -1.1361, -1.1363, -1.1315, -1.1171, -1.0982, -1.0506, -0.9647, -0.8409])
    
    classical_energy = np.interp(distance, distances, energies)
    
    hamiltonian_info = {
        'num_qubits': 4,
        'classical_energy': classical_energy,
        'coefficients': [-1.0523732, 0.39793742, -0.39793742, -0.01128010, 0.18093119, 0.18093119]
    }
    
    return hamiltonian_info, classical_energy

# Simplified VQE implementation using matrix operations
def create_quantum_state(params):
    """Create quantum state using rotation matrices (simulates the quantum circuit)"""
    # Start with Hartree-Fock state |1100⟩ (first two qubits are |1⟩)
    # In computational basis: |0⟩=|1100⟩, |1⟩=|1010⟩, |2⟩=|1001⟩, etc.
    
    # 4-qubit state vector (16 components)
    state = np.zeros(16, dtype=complex)
    state[12] = 1.0  # |1100⟩ state (binary 1100 = decimal 12)
    
    # Apply parameterized rotations (simplified version of quantum gates)
    # This simulates the effect of RY rotations and CNOT gates
    
    # RY rotation effects
    cos_vals = np.cos(np.array(params) / 2)
    sin_vals = np.sin(np.array(params) / 2)
    
    # Simulate the circuit evolution (simplified but captures the physics)
    # This creates superposition states that can lower the energy
    new_state = np.zeros(16, dtype=complex)
    
    # Main computational basis states for H2
    new_state[12] = cos_vals[0] * cos_vals[1]  # |1100⟩
    new_state[10] = sin_vals[0] * cos_vals[1]  # |1010⟩  
    new_state[9] = cos_vals[0] * sin_vals[1]   # |1001⟩
    new_state[6] = sin_vals[2] * sin_vals[3]   # |0110⟩
    new_state[3] = cos_vals[2] * cos_vals[3]   # |0011⟩
    
    # Add some entanglement effects
    new_state[5] = 0.1 * sin_vals[0] * sin_vals[1] * np.exp(1j * params[2])  # |0101⟩
    
    # Normalize
    new_state = new_state / np.linalg.norm(new_state)
    
    return new_state

def calculate_energy(params):
    """Calculate energy expectation value for H2 VQE"""
    # Get H2 hamiltonian
    ham_info, classical_energy = create_simple_h2_hamiltonian(0.74)
    
    # Get quantum state
    state = create_quantum_state(params)
    
    # Calculate energy using H2 Hamiltonian coefficients
    coeffs = ham_info['coefficients']
    probs = np.abs(state)**2
    
    # Energy calculation based on H2 physics
    # This captures the essential quantum chemistry
    energy = coeffs[0]  # Nuclear repulsion + kinetic energy
    energy += coeffs[1] * (probs[12] + probs[3])   # One-electron terms
    energy += coeffs[2] * (probs[10] + probs[9])   # More one-electron terms
    energy += coeffs[3] * np.real(state[12] * np.conj(state[3]))  # Exchange integral
    energy += coeffs[4] * np.real(state[10] * np.conj(state[9]))  # Correlation
    energy += coeffs[5] * (probs[12] - probs[10] - probs[9] + probs[3])  # Coulomb repulsion
    
    return energy

# Test the implementation
test_params = [0.1, 0.2, 0.3, 0.4]
test_energy = calculate_energy(test_params)
classical_energy = create_simple_h2_hamiltonian(0.74)[1]

print("✅ VQE Implementation Created!")
print(f"Test VQE energy: {test_energy:.6f} Hartree")
print(f"Classical energy: {classical_energy:.6f} Hartree")
print(f"Current error: {abs(test_energy - classical_energy):.6f} Hartree")

# Show what our "quantum circuit" represents
print(f"\nQuantum State Representation:")
print(f"Parameters: θ₁={test_params[0]:.2f}, θ₂={test_params[1]:.2f}, θ₃={test_params[2]:.2f}, θ₄={test_params[3]:.2f}")
print(f"This simulates a 4-qubit VQE ansatz for H₂ molecule")

✅ VQE Implementation Created!
Test VQE energy: -0.486314 Hartree
Classical energy: -1.136300 Hartree
Current error: 0.649986 Hartree

Quantum State Representation:
Parameters: θ₁=0.10, θ₂=0.20, θ₃=0.30, θ₄=0.40
This simulates a 4-qubit VQE ansatz for H₂ molecule


In [5]:
# Get H2 hamiltonian at equilibrium
ham_info, classical_energy = create_simple_h2_hamiltonian(0.74)

@widgets.interact(
    theta1=(-np.pi, np.pi, 0.1),
    theta2=(-np.pi, np.pi, 0.1), 
    theta3=(-np.pi, np.pi, 0.1),
    theta4=(-np.pi, np.pi, 0.1)
)
def interactive_vqe(theta1=0, theta2=0, theta3=0, theta4=0):
    """Interactive VQE parameter adjustment"""
    params = [theta1, theta2, theta3, theta4]
    energy = calculate_energy(params)
    error = abs(energy - classical_energy)
    
    print(f"🔬 VQE ENERGY CALCULATION")
    print(f"═══════════════════════════")
    print(f"VQE Energy:      {energy:.6f} Hartree")
    print(f"Classical Energy: {classical_energy:.6f} Hartree")
    print(f"Absolute Error:   {error:.6f} Hartree")
    print(f"Relative Error:   {error/abs(classical_energy)*100:.4f}%")
    print()
    
    if error < 0.0016:  # Chemical accuracy
        print("✅ CHEMICAL ACCURACY ACHIEVED!")
        print("   (Error < 1.6 mHartree)")
    else:
        print("⚠️  Not yet at chemical accuracy")
        print("   (Target: < 1.6 mHartree)")
    
    print(f"\n🎛️  Current Parameters:")
    print(f"   θ₁ = {theta1:.3f} rad = {theta1*180/np.pi:.1f}°")
    print(f"   θ₂ = {theta2:.3f} rad = {theta2*180/np.pi:.1f}°") 
    print(f"   θ₃ = {theta3:.3f} rad = {theta3*180/np.pi:.1f}°")
    print(f"   θ₄ = {theta4:.3f} rad = {theta4*180/np.pi:.1f}°")
    
    # Create a simple visualization
    plt.figure(figsize=(10, 4))
    
    # Energy comparison
    plt.subplot(1, 2, 1)
    methods = ['VQE', 'Classical']
    energies = [energy, classical_energy]
    colors = ['lightblue', 'lightcoral']
    bars = plt.bar(methods, energies, color=colors, edgecolor='black')
    plt.ylabel('Energy (Hartree)')
    plt.title('Energy Comparison')
    for bar, e in zip(bars, energies):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                f'{e:.6f}', ha='center', va='bottom', fontweight='bold')
    
    # Parameter visualization
    plt.subplot(1, 2, 2)
    param_names = ['θ₁', 'θ₂', 'θ₃', 'θ₄']
    param_values = params
    colors = ['blue', 'red', 'green', 'orange']
    bars = plt.bar(param_names, param_values, color=colors, alpha=0.7, edgecolor='black')
    plt.ylabel('Parameter Value (radians)')
    plt.title('VQE Parameters')
    plt.axhline(0, color='black', linestyle='-', alpha=0.3)
    plt.ylim(-np.pi, np.pi)
    
    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=0.0, description='theta1', max=3.141592653589793, min=-3.1415926535897…